<a href="https://colab.research.google.com/github/vardhaman3/ML-PROJECT/blob/vardhaman/Copy_of_ML_major_project_smart_knower_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
 
#Completed By
# 1. Srijan Prem Srivastava: Email- srijansriv@gmail.com
# 2. G Vardhaman: Email- gajjalavardhaman99@gmail.com
# 3. Ketki Churi: Email- ket28me@gmail.com
# 4. Anand Krishnan B: Email- krisanofficial21@gmail.com

In [ ]:
!pip install vaderSentiment

     |████████████████████████████████| 133kB 2.7MB/s 


In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
vs=SentimentIntensityAnalyzer()

In [ ]:

import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import os

In [ ]:
urls= ['https://inshorts.com/en/read/sports',
       'https://inshorts.com/en/read/world',
       'https://inshorts.com/en/read/politics']

In [ ]:
def build_dataset(urls):    
  news_data= []
  for url in urls:
    news_category=url.split('/')[-1]
    data=requests.get(url)
    soup= BeautifulSoup(data.content)

    news_articles=[{'news_headline':headline.find('span',attrs={"itemprop":"headline"}).string,
                    'news_article':article.find('div',attrs={"itemprop":"articleBody"}).string,
                    'news_category':news_category}
                   
                   for headline,article in zip(soup.find_all('div',class_=["news-card-title news-right-box"]),
                                               soup.find_all('div',class_=["news-card-content news-right-box"]))]
    news_articles =news_articles[0:20]
    news_data.extend(news_articles)
  df=pd.DataFrame(news_data)
  return df

In [ ]:
df=build_dataset(urls)
df.head()

In [ ]:
df.to_csv('news.csv',index=False)

In [ ]:
import pandas as pd
df=pd.read_csv('/content/news.csv')
df

In [ ]:
#Pre-processing 
import nltk
nltk.download('stopwords')
stopword_list = nltk.corpus.stopwords.words('english')
stopword_list.remove('not')
stopword_list.remove('no')
len(stopword_list)

In [ ]:
def html_tag(text):
  soup=BeautifulSoup(text,"html.parser")
  new_text=soup.get_text()
  return new_text


In [ ]:
!pip install contractions
import contractions
def con(text):
  expand= contractions.fix(text)
  return expand

In [ ]:
import re
def remove_sp(text):
  pattern = r'[^A-Za-z0-9\s]'
  text= re.sub(pattern,'',text)
  return text

In [ ]:
from nltk.tokenize.toktok import ToktokTokenizer
tokenizer = ToktokTokenizer()
def remove_stopwords(text):
  tokens= tokenizer.tokenize(text)
  tokens= [token.strip() for token in tokens]
  filtered_tokens=[token for token in tokens if token not in stopword_list]
  filtered_text=' '.join(filtered_tokens)
  return filtered_text

In [ ]:
df.news_headline=df.news_headline.apply(lambda x:x.lower())
df.news_article=df.news_article.apply(lambda x:x.lower())

df.news_headline=df.news_headline.apply(html_tag)
df.news_article=df.news_article.apply(html_tag)

df.news_headline=df.news_headline.apply(con)
df.news_article=df.news_article.apply(con)

df.news_headline=df.news_headline.apply(remove_sp)
df.news_article=df.news_article.apply(remove_sp)

df.news_headline=df.news_headline.apply(remove_stopwords)
df.news_article=df.news_article.apply(remove_stopwords)

In [ ]:
df['compound']= df['news_article'].apply(lambda x: vs.polarity_scores(x)['compound'])

In [ ]:
df.loc[df['compound'] >= 0.5, 'Sentiment'] = 'Positive'
df.loc[df['compound'] <= -0.5, 'Sentiment'] = 'Negative'
df.loc[(df['compound'] > -0.5) & (df['compound'] <0.5), 'Sentiment'] = 'Neutral'

In [ ]:
df

In [ ]:
#Deployment
!pip install streamlit
!pip install pyngrok
from pyngrok import ngrok

     |████████████████████████████████| 7.4MB 2.7MB/s 
     |████████████████████████████████| 6.7MB 44.5MB/s 
     |████████████████████████████████| 133kB 44.1MB/s 
     |████████████████████████████████| 163kB 42.3MB/s 
     |████████████████████████████████| 102kB 9.7MB/s 
     |████████████████████████████████| 4.5MB 41.7MB/s 
     |████████████████████████████████| 112kB 35.3MB/s 
     |████████████████████████████████| 133kB 45.3MB/s 
     |████████████████████████████████| 71kB 6.9MB/s 
     |████████████████████████████████| 71kB 7.0MB/s 
     |████████████████████████████████| 122kB 37.9MB/s 
  Created wheel for watchdog: filename=watchdog-0.10.3-cp36-none-any.whl size=73873 sha256=addd2b2e6206a9d74dcd4921d7143c927f1024539d5ad533b1785398b52af61c
  Stored in directory: /root/.cache/pip/wheels/a8/1d/38/2c19bb311f67cc7b4d07a2ec5ea36ab1a0a0ea50db994a5bc7
  Created wheel for blinker: filename=blinker-1.4-cp36-none-any.whl size=13450 sha256=3916ae116b4f2214b22809acdd06e244c361c2393

  Created wheel for pyngrok: filename=pyngrok-5.0.0-cp36-none-any.whl size=18780 sha256=cfd7e9d2de9bf63bc0c1b652a14f889ab5811be8f7db39433120316c9b058054
  Stored in directory: /root/.cache/pip/wheels/95/df/23/af8dde08c3fcdc7b966adcacef48ab29aa3b0b1860df5d2b79
Successfully built pyngrok


In [ ]:
%%writefile app.py
import streamlit as st
import numpy as np
from skimage.io import imread
from skimage.transform import resize
st.title('Sentiment analysis using vader sentiment')

Writing app.py


In [ ]:
!nohup streamlit run app.py &
url=ngrok.connect(port='8501')
url